<a href="https://colab.research.google.com/github/himanshunaidu/human_or_horse/blob/master/HumanHorse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import math
import zipfile
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPool2D, Flatten, Dense, LeakyReLU, Dropout
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from PIL import Image, ImageOps

In [17]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


#Loading Dataset

In [2]:
#TRAINING SET
!wget --no-check-certificate \
https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
-O /tmp/horse-or-human.zip

--2021-08-26 16:24:31--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.128, 142.250.141.128, 142.251.2.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M   294MB/s    in 0.5s    

2021-08-26 16:24:32 (294 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]



In [3]:
dataset_zip_path = '/tmp/horse-or-human.zip'
dataset_ref = zipfile.ZipFile(dataset_zip_path)
dataset_ref.extractall('/tmp/horse-or-human')
dataset_ref.close()

In [4]:
train_human_dir = '/tmp/horse-or-human/humans'
train_horse_dir = '/tmp/horse-or-human/horses'

In [5]:
train_human_names = os.listdir(train_human_dir)
print(len(train_human_names))
train_horse_names = os.listdir(train_horse_dir)
print(len(train_horse_names))

527
500


In [ ]:
#Experiment to find the dimensions
min_height, min_width = math.inf, math.inf
max_height, max_width = -1, -1
avg_height, avg_width = 0, 0

for i, path in enumerate(validation_human_names):
  image_path = os.path.join(validation_human_dir, path)
  img = Image.open(image_path)
  width, height = img.size

  min_height, min_width = min(min_height, height), min(min_width, width)
  max_height, max_width = max(min_height, height), max(min_width, width)
  avg_height, avg_width = avg_height+height, avg_width+width

avg_height, avg_width = avg_height//len(validation_human_names), avg_width//len(validation_human_names)
print(min_height, min_width)
print(max_height, max_width)
print(avg_height, avg_width)

In [7]:
#VALIDATION SET
!wget --no-check-certificate \
https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
-O /tmp/validation-horse-or-human.zip

--2021-08-26 16:25:21--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.128, 142.250.141.128, 142.251.2.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/zip]
Saving to: ‘/tmp/validation-horse-or-human.zip’

/tmp/validation-hor 100%[===================>]  10.95M  58.1MB/s    in 0.2s    

2021-08-26 16:25:21 (58.1 MB/s) - ‘/tmp/validation-horse-or-human.zip’ saved [11480187/11480187]



In [8]:
dataset_validation_zip_path = '/tmp/validation-horse-or-human.zip'
dataset_validation_ref = zipfile.ZipFile(dataset_zip_path)
dataset_validation_ref.extractall('/tmp/validation-horse-or-human')
dataset_validation_ref.close()

In [9]:
validation_human_dir = '/tmp/validation-horse-or-human/humans'
validation_horse_dir = '/tmp/validation-horse-or-human/horses'

In [10]:
validation_human_names = os.listdir(validation_human_dir)
print(len(validation_human_names))
validation_horse_names = os.listdir(validation_horse_dir)
print(len(validation_horse_names))

527
500


#Data Pre-processing

In [11]:
train_datagen = image.ImageDataGenerator(rescale=1/255.0)
train_generator = train_datagen.flow_from_directory('/tmp/horse-or-human', target_size=(300, 300), color_mode='rgb',\
                                                    class_mode='binary', batch_size=128)

Found 1027 images belonging to 2 classes.


In [12]:
validation_datagen = image.ImageDataGenerator(rescale=1/255.0)
validation_datagen = train_datagen.flow_from_directory('/tmp/validation-horse-or-human', target_size=(300, 300), \
                                                       color_mode='rgb', class_mode='binary', batch_size=128)

Found 1027 images belonging to 2 classes.


#Create Model

In [13]:
#params
alpha = 0.15
dropout_rate = 0.3

In [14]:
#Model
i = Input(shape=(300, 300, 3))

#Convolution 1
x = Conv2D(32, (3, 3), strides=(1, 1), padding='valid', activation=LeakyReLU(alpha=alpha))(i)
x = BatchNormalization()(x)
x = MaxPool2D((2, 2), strides=(2, 2))(x)

#Convolution 2
x = Conv2D(32, (3, 3), strides=(1, 1), padding='valid', activation=LeakyReLU(alpha=alpha))(x)
x = BatchNormalization()(x)
x = MaxPool2D((2, 2), strides=(2, 2))(x)

#Convolution 3
x = Conv2D(64, (3, 3), strides=(1, 1), padding='valid', activation=LeakyReLU(alpha=alpha))(x)
x = BatchNormalization()(x)
x = MaxPool2D((2, 2), strides=(2, 2))(x)

x = Flatten()(x)

x = Dense(1225, activation=LeakyReLU(alpha=alpha))(x)
x = Dropout(rate=dropout_rate)(x)
x = Dense(100, activation=LeakyReLU(alpha=alpha))(x)
x = Dropout(rate=dropout_rate)(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(i, x)

In [15]:
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 300, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 298, 298, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 298, 298, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 147, 147, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0     

In [16]:
#Compile
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [22]:
save_path = "gdrive/My Drive/Colab Notebooks/HH1"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=save_path, save_weights_only=True, verbose=1)

In [ ]:
# history = model.fit_generator(train_generator, steps_per_epoch=8, epochs=15, )